In [70]:
import os
import time
import pathlib
import random
import io
import re

import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from playsound import playsound
from time import perf_counter
from datetime import timedelta
from zipfile import ZipFile
from collections import Counter

this_directory = pathlib.Path().absolute()

In [71]:
#SETUP DRIVER
RUN_SELENIUM = True
testing = False

download_path = str(this_directory)+'\\NewsData'

# Set Chrome options
options = webdriver.ChromeOptions()

options.headless = False
options.add_argument('--start-maximized')
options.add_experimental_option("prefs", {
    "download.default_directory": download_path,
    "download.prompt_for_download": False,
})
options.add_argument(f"user-data-dir={this_directory}\\cookies")
options.add_argument('--disable-features=NetworkService')

# Initialize the Chrome webdriver
if RUN_SELENIUM:
    driver = webdriver.Chrome(options=options)

In [72]:
def interactXP(driver, dom, selector, assignment, fill="", enter=False):
    ''' click or sendkeys to an element of the driver By XPATH
        build the XPATH of dom, selector, and assignment
        if fill, send keys with value of fill; otherwise just click
        press ENTER/RETURN if enter
    '''
    path_str = "//" +\
                dom+\
                "[@"+\
                selector+\
                "='"+\
                assignment+\
                "']"
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.XPATH,path_str)))

    element = driver.find_element(By.XPATH, path_str)
    if fill == "":
        element.click()
    else:
        try:
            element.send_keys(Keys.CONTROL+"a"+ Keys.DELETE)
        except:
            pass
        element.send_keys(fill)
    
    if enter:
        element.send_keys(Keys.ENTER)
    
    return

In [73]:
def login_page(driver,creds):
    # Enter the desired username into the input field
    interactXP(driver,'input','id','username',fill=creds['user'])

    # Enter the desired password into the input field, and submit
    interactXP(driver,'input','id','password',fill=creds['pass'], enter=True)
    return

In [74]:
def wait_for_auth(driver):
    try: 
        current_url = driver.current_url
        auth_page = driver.find_element(By.XPATH,'//iframe[@id="duo_iframe"]') 
        time.sleep(3)

        if current_url == driver.current_url:
            playsound('alerted.wav')
            print('---Need Authention---')
            playsound('alerted.wav')

            #Give user 90 seconds to authenticate
            wait90 = WebDriverWait(driver, 90)
            wait90.until(EC.url_changes(current_url))

            if current_url == driver.current_url:
                print('user failed to authenticate')
            else:
                print('authentication success')

    except Exception as e:
        print('no authentication page')
    
    return

In [75]:
def fill_filters(driver, start_str="10/07/2022", end_str="11/07/2022"):
    print("filling dates:",start_str,", ",end_str,"...")
    
    misbehaving = 0
    wait = WebDriverWait(driver, 10)
    time.sleep((random.random()*2) +1)
    
    wait.until(EC.presence_of_element_located((By.XPATH,'//button[@data-filtertype="datestr-news"]')))
    
    while misbehaving < 5:
        interactXP(driver, 'button','data-filtertype',"datestr-news")
        try:
            wait.until(EC.presence_of_element_located((By.XPATH,"//input[@aria-label='Input Min Date']")))
            misbehaving=100
        except:
            #fail to load submenu -> click to close, loop to reopen
            interactXP(driver, 'button','data-filtertype',"datestr-news")
            time.sleep(round(random.random()*2,1))
            print('submenu misbehaving; retrying')
            misbehaving += 1

    time.sleep(1)
    interactXP(driver, 'input','aria-label','Input Min Date', fill=start_str)
    interactXP(driver, 'input','aria-label','Input Max Date', fill=end_str)
    time.sleep(1)

    end_field = driver.find_element(By.XPATH, '//input[@aria-label="Input Max Date"]')
    end_field.send_keys('\t')
    cal = driver.switch_to.active_element
    cal.send_keys('\t')
    button = driver.switch_to.active_element
    button.click()

    time.sleep((random.random()*2) +1)

    results = driver.find_element(By.XPATH, '//header[@class="resultsHeader"]')
    res_info = results.find_elements(By.XPATH,'.//h2')[0].find_elements(By.XPATH,'.//span')[0].text
    max_n = "".join(res_info.split('(')[1].split(')')[0].split('+')[0].split(','))
    
    max_n = "1000" if int(max_n) >=1000 else max_n

    misbehaving = 0
    found = False
    
    if max_n == 0:
        raise Exception('0 results')
    
    if int(max_n) > 100: # filter out subject
        while misbehaving < 5:
            interactXP(driver,'button','data-filtertype',"en-subject")
            try:
                wait.until(EC.presence_of_element_located((By.XPATH, '//input[@data-value="Government & Public Administration"]')))
                misbehaving=100
                found =True
            except:
                #fail to load submenu -> click to close, loop to reopen
                interactXP(driver,'button','data-filtertype',"en-subject")
                time.sleep(round(random.random()*2,1))
                print('submenu misbehaving; retrying')
                misbehaving +=1
    
    if found:
        #interact with unlabelled parent element
        gov = driver.find_element(By.XPATH, '//input[@data-value="Government & Public Administration"]')
        gov_li = gov.find_element(By.XPATH,"./..")
        gov_li.click()
    
    return  found


In [76]:
def download1000(driver,fpath,fname,max_n):
    print('downloading...')
    time.sleep((random.random()*12) +10)
    
    down = driver.find_element(By.XPATH, '//button[@data-label="Download "]')
    down_li = down.find_element(By.XPATH,"..")
    down_li.click()
    time.sleep(0.5)
    interactXP(driver,'input','id',"ResultsListOnly")
    time.sleep(0.5)
    interactXP(driver,'input','id',"XLSX")
    time.sleep(0.5)
    interactXP(driver,'input','id','FileName', fill=fname)
    time.sleep(0.5)
    input_elem = driver.find_elements(By.XPATH, '//input[@id="SelectedRange"]')[1]
    input_elem.clear()
        
    input_elem.send_keys('1-'+max_n) #gotta get the max number

    interactXP(driver,'button','data-action','download')
    wait_c = 0
    
    while not os.path.exists(os.path.join(fpath,fname+'.ZIP')):
        fail = driver.find_elements(By.XPATH, '//div[contains(@class, "validation")]')
        if len(fail) > 0 or wait_c > 25:
            raise Exception('Download screw up')
        time.sleep(1)
        wait_c += 1

        
    print('successfully saved',fname+'.ZIP')
    
    return os.path.join(fpath,fname+'.ZIP')
    #driver.switch_to.default_content()
    

In [77]:
def date_to_str(date,conn):
    return str(date['m'])+conn +str(date['d'])+conn +str(date['y'])
    
def alter_date(date,elem,n):
    d = date.copy()
    d[elem] = d[elem] + n
    return  d

In [78]:
def access_page(driver, wait, creds, target):
    if "shibboleth" in driver.current_url:
        print('logging in...')
        login_page(driver,creds)
        wait.until(EC.presence_of_all_elements_located)
        
        wait_for_auth(driver)
        wait.until(EC.presence_of_all_elements_located)
    time.sleep(10)

    if "bisacademicresearchhome" in driver.current_url:
        print('redirecting...')
        try:
            elem = driver.find_element(By.XPATH,'//lng-search-input')
            elem.click() 
        except:
            pass
        elem = driver.find_element(By.XPATH,'//lng-expanding-textarea') 
        elem.send_keys(target)
        elem.send_keys(Keys.ENTER)

    wait.until(EC.presence_of_element_located((By.XPATH, '//textarea[@id="searchTerms"]')))
    return

In [79]:
dated_df = pd.read_excel("Elections/ElectionDates.xlsx")
dated_df = dated_df[['Year','Date']]
dated_df['Date'] = pd.to_datetime(dated_df.Date).dt.date
dated = dict(zip(dated_df.Year, dated_df.Date))

def fetch_preprocess_targets(f, dated=dated):
    def perc(row):
        if row['totalvotes'] == 0:
            return 0
        return row['candidatevotes'] / row['totalvotes']

    df = pd.read_csv(f,encoding = "ISO-8859-1")

    if 'president' in f:
        df['district'] = ['nationwide']*len(df)
    df['percentvote'] = df.apply(lambda row: perc(row), axis=1)

    df['date'] = df.apply(lambda row: dated[row['year']], axis=1)
    
    if 'party_detailed' in df.columns:
        df['party'] = df['party_detailed']
    if 'runoff' in df.columns:
        df = df[df['runoff'] !=True]    # ==False here was dropping nearly 8k elections
    if 'special' in df.columns:
        df = df[df['special'] ==False] 
    if 'stage' in df.columns:
        df = df[df['stage'].str.lower() == 'gen']
    
    df = df.drop(['version',
                  'notes',
                  'writein',
                  'unofficial',
                  'party_detailed',
                  'party_simplified', 
                  'runoff',
                  'special',
                  'mode',
                  'stage',
                  'fusion_ticket'], axis=1, errors='ignore')    #state_fips	state_cen	state_ic?
    
    #Drop years before 2000, runoffs and special elections; and candidates who didn't finish in top 2
    df = df[~(df['year'] < 1999)]  
    df = df.sort_values(['year',
                         'state',
                         'district',
                         'totalvotes',
                         'percentvote'],ascending=False).groupby(['year',
                                                                  'state',
                                                                  'district',
                                                                  'totalvotes'])#.head(2).reset_index(drop=True) 
    
    grouped_dfs = []
    for name, group in df:
        tmp = group.head(2)
        if tmp['percentvote'].sum() >= 0.70:
            grouped_dfs.append(tmp)
        else:
            grouped_dfs.append(group.head(3))
    
    df =pd.concat(grouped_dfs)
    df = df[df['percentvote'] >= 0.02]
    df = df[df['percentvote'] <= 0.99]

    return df

In [80]:
def main_loop(driver, wait, target, elect_date, creds):
    wait.until(EC.presence_of_all_elements_located)
    print("Researching",target,'...')
    
    interactXP(driver,'textarea','id','searchTerms',fill=target,enter=True)
    wait.until(EC.presence_of_all_elements_located)

    access_page(driver,wait,creds,target)

    found_government = fill_filters(driver, 
                start_str=  date_to_str(alter_date(alter_date(elect_date,'d',-1),'m',-1),'/'), 
                end_str=    date_to_str(alter_date(elect_date,'d',-1),'/'))

    wait.until(EC.presence_of_all_elements_located)
    save_file = target+"_"+ date_to_str(elect_date,'-')

    wait.until(EC.presence_of_element_located((By.XPATH,'//header[@class="resultsHeader"]')))
    time.sleep(2)
    
    results = driver.find_element(By.XPATH, '//header[@class="resultsHeader"]')
    res_info = results.find_elements(By.XPATH,'.//h2')[0].find_elements(By.XPATH,'.//span')[0].text
    max_n = "".join(res_info.split('(')[1].split(')')[0].split('+')[0].split(','))
    
    max_n = "1000" if int(max_n) >=1000 else max_n

    filters = driver.find_elements(By.XPATH, '//ul[@class="filters-used "]/li')

    if len(filters) < 1 or  (len(filters) < 2 and found_government):
        raise Exception('Filters not entered properly')
    if max_n == "1":
        raise Exception("there's only 1 result, can't download properly")
        
    save_file = download1000(driver, download_path, save_file, max_n)
    return save_file

In [81]:
def read_zip(zip_fn, extract_fn=None):
    zf = ZipFile(zip_fn)
    if extract_fn:
        return zf.read(extract_fn)
    else:
        return {name:zf.read(name) for name in zf.namelist()}

In [82]:
print('loading credentials...')
with open('creds.txt', 'r') as c:
    raw_cred = c.read().split('\n')
creds = {'user':raw_cred[0],'pass':raw_cred[1], 'url': raw_cred[2]}

timeout = 10
wait = WebDriverWait(driver, timeout)

if RUN_SELENIUM:
    tprime = perf_counter()
    print('Accessing Nexis...')
    print(creds['url'])
    driver.get(creds['url'])
    
    access_page(driver,wait,creds, "Basic")

    #here's where the looping goes
    #get a target and a date from the data in Elections
    all_elect = []
    for fil in ['1976-2020-president.csv','1976-2020-senate.csv','1976-2020-house.csv']:
        all_elect.append(fetch_preprocess_targets('Elections/'+fil, dated))

    df_all = pd.concat(all_elect).reset_index(drop=True)
    print(len(df_all))

    cc = 0
    for id,row in df_all.iterrows():
        if "signin" in driver.current_url:
            time.sleep(random.randint(25,50))
            driver.get(creds['url'])
            time.sleep(random.randint(25,50))
            access_page(driver,wait,creds, "Basic")

        if os.path.isfile('failures.txt'):
            if len(open("successes.txt",'r').readlines()) + len(open('failures.txt','r').readlines()) >= len(df_all):
                print("might be done")
                break
            

        completed_files = os.listdir('NewsData')
        t0 = perf_counter()
        
        if row['office'] == "US PRESIDENT":
            target = row['candidate'] + " "+ row['state']
        else:
            target = row['candidate']
        
        dateI = row['date']
        elect_date = {'d':dateI.day,'m':dateI.month,'y':dateI.year}
        target2 = re.sub(r'[^A-Za-z0-9 ]+','',target)

        
        cc += 1
        if (target+"_"+date_to_str(elect_date,'-')+".ZIP" not in completed_files and \
            target2+"_"+date_to_str(elect_date,'-')+".ZIP" not in completed_files):
            time.sleep(2)
            if target2 != target:
                target = target2
            print(cc)
            try:
                out_file = main_loop(driver,wait,target,elect_date,creds)
                
                duration = perf_counter() - t0
                td = str(timedelta(seconds=duration))

                with open('successes.txt','a') as f:
                    f.write(target+ ", " +date_to_str(elect_date,'-')+"  "+ td+"\n")
                print(td)
            except Exception as e:
                e_str ="  ".join(str(e).split('\n'))
                issue = target+ ", " +date_to_str(elect_date,'-')+' FAIL '+e_str[:35]
                print(e)
                with open('failures.txt','a') as f:
                    f.write(issue +'\n')
                print(issue)
    duration = perf_counter() - t0
    ttot = str(timedelta(seconds=duration))
    print(len(df_all), len(completed_files))
    print(ttot)
    

loading credentials...
Accessing Nexis...
https://guides.library.brandeis.edu/nexis
logging in...
no authentication page
redirecting...
10552
324
Researching ROMNEY MITT DISTRICT OF COLUMBIA ...
filling dates: 10/5/2012 ,  11/5/2012 ...
downloading...
successfully saved ROMNEY MITT DISTRICT OF COLUMBIA_11-6-2012.ZIP
0:00:52.409991
340
Researching OBAMA BARACK H KANSAS ...
filling dates: 10/5/2012 ,  11/5/2012 ...
there's only 1 result, can't download properly
OBAMA BARACK H KANSAS, 11-6-2012 FAIL there's only 1 result, can't downlo
425
Researching CLINTON HILLARY DISTRICT OF COLUMBIA ...
filling dates: 10/7/2016 ,  11/7/2016 ...
downloading...
successfully saved CLINTON HILLARY DISTRICT OF COLUMBIA_11-8-2016.ZIP
0:00:47.418776
426
Researching TRUMP DONALD J DISTRICT OF COLUMBIA ...
filling dates: 10/7/2016 ,  11/7/2016 ...
downloading...
successfully saved TRUMP DONALD J DISTRICT OF COLUMBIA_11-8-2016.ZIP
0:00:58.758006
450
Researching TRUMP DONALD J MARYLAND ...
filling dates: 10/7/20

KeyboardInterrupt: 

In [ ]:
if testing:
    df = pd.read_excel(io.BytesIO(read_zip(out_file, 'Results list for_'+target+'.xlsx')))
    df['Date']= pd.to_datetime(df['Date'])
    df = df.sort_values(by='Date')
    df

In [ ]:
if testing:
    def perc(row):
        if row['totalvotes'] == 0:
            return 0
        return row['candidatevotes'] / row['totalvotes']
    dfp = pd.read_csv('Elections/1976-2020-president.csv',encoding = "ISO-8859-1")
    dfs = pd.read_csv('Elections/1976-2020-senate.csv',encoding = "ISO-8859-1")
    dfh = pd.read_csv('Elections/1976-2020-house.csv',encoding = "ISO-8859-1")
    dfh['percentvote'] = dfh.apply(lambda row: perc(row), axis=1)
    dfs['percentvote'] = dfs.apply(lambda row: perc(row), axis=1)
    dfp['percentvote'] = dfp.apply(lambda row: perc(row), axis=1)

In [ ]:
if testing: 
    print(Counter(df_all['party']).most_common(50))
    
    for idx, row in df_all[df_all['percentvote'] < 0.1].iterrows():
        tmp = df_all[(df_all['year'] ==row['year']) & (df_all['state'] ==row['state']) & (df_all['district'] ==row['district']) & (df_all['totalvotes'] ==row['totalvotes'])]
        if tmp['percentvote'].sum() <= 0.85:
            print(tmp)

In [ ]:
df_all

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,candidatevotes,totalvotes,district,percentvote,date,party
0,2000,ALABAMA,AL,1,63,41,US PRESIDENT,"BUSH, GEORGE W.",941173,1666272,nationwide,0.564838,2000-11-07,REPUBLICAN
1,2000,ALABAMA,AL,1,63,41,US PRESIDENT,"GORE, AL",692611,1666272,nationwide,0.415665,2000-11-07,DEMOCRAT
2,2000,ALASKA,AK,2,94,81,US PRESIDENT,"BUSH, GEORGE W.",167398,285560,nationwide,0.586210,2000-11-07,REPUBLICAN
3,2000,ALASKA,AK,2,94,81,US PRESIDENT,"GORE, AL",79004,285560,nationwide,0.276663,2000-11-07,DEMOCRAT
4,2000,ARIZONA,AZ,4,86,61,US PRESIDENT,"BUSH, GEORGE W.",781652,1532016,nationwide,0.510211,2000-11-07,REPUBLICAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5108,2020,WISCONSIN,WI,55,35,25,US HOUSE,TRICIA ZUNKER,162741,415007,7,0.392140,2020-11-03,DEMOCRAT
5109,2020,WISCONSIN,WI,55,35,25,US HOUSE,MIKE GALLAGHER,268173,417838,8,0.641811,2020-11-03,REPUBLICAN
5110,2020,WISCONSIN,WI,55,35,25,US HOUSE,AMANDA STUCK,149558,417838,8,0.357933,2020-11-03,DEMOCRAT
5111,2020,WYOMING,WY,56,83,68,US HOUSE,LIZ CHENEY,185732,278503,0,0.666894,2020-11-03,REPUBLICAN


In [ ]:
filters = driver.find_elements(By.XPATH, '//ul[@class="filters-used "]/li')

In [ ]:
filters

[<selenium.webdriver.remote.webelement.WebElement (session="8836107f7bbde5c3e2c3ef05b5f0d7d9", element="d4821ab5-3d46-4a13-a6d3-eb9eaf0caf24")>]